# ESTIMATING "RELATIVE PHYSICAL FATIGUE INDEX" OF PLAYERS USING ICE HOCKEY METRICS

#### **Objective:** To develop an application that enables coaches to input match data and obtain instant feedback on potential fatigue levels,based on the dataset's metrics.

#### **Functionality:** Upload data files and obtain instant feedback on potential fatigue levels.

#### **Applications:**

1. **Player Management and Rotation:**
   Use the Fatigue Index (FI) to guide real-time player rotation decisions, optimizing performance and minimizing injury risks.

2. **Injury Prevention in Training:**
   Incorporate FI to tailor training intensity and recovery, reducing the likelihood of player injuries.

3. **Performance Analysis and Strategy:**
   Integrate FI with performance metrics to refine game strategies, making informed in-game decisions for optimal outcomes.


### DATA LOADING AND PREPROCESSING


In [1052]:
# Import required Libraries
import pandas as pd

import matplotlib.pyplot as plt

In [1053]:
# Load the Original Datasets
lugano = pd.read_csv("LUGANO-ORIG.csv", encoding="ISO-8859-1", sep=";")
zsc = pd.read_csv("ZHC-ORIG.csv", encoding="ISO-8859-1", sep=";")

In [1054]:
# We change the delimiter to ',' rename and save the file for ease of use
lugano.to_csv("lhc-lugano.csv", sep=",", index=False)
zsc.to_csv("lhc-zsc.csv", sep=",", index=False)

In [1055]:
# Load the updated datasets into Pandas DataFrame
df_lugano = pd.read_csv("lhc-lugano.csv", encoding="ISO-8859-1")
df_zsc = pd.read_csv("lhc-zsc.csv", encoding="ISO-8859-1")

#### Handling missing values

#### Filtering the dataset to include only the relevant columns

In [1056]:
# List of columns to keep in the dataset
columns_to_keep = [
    "Group name",
    "Position",
    "Description",
    "Duration (s)",
    "Distance (m)",
    "Time on Ice (s)",
    "High Metabolic Power Distance (m)",
    "Mechanical Intensity",
]


# Function to filter the columns
def filter_columns(df, columns_to_keep):
    """
    Filters a DataFrame to only include specified columns.

    Parameters:
    - df (pd.DataFrame): The original DataFrame.
    - columns_to_keep (list of str): List of column names to keep.

    Returns:
    - pd.DataFrame: A new DataFrame with only the specified columns.
    """
    return df[columns_to_keep]


# Example usage:
df_imp_lugano = filter_columns(df_lugano, columns_to_keep)
df_imp_zsc = filter_columns(df_zsc, columns_to_keep)

# Print the first 5 rows of the datasets
print(df_imp_lugano.head())

print(
    "===================================================================================================="
)

print(df_imp_zsc.head())

  Group name Position Description  Duration (s)  Distance (m)  \
0    Defense        D   vs Lugano          8836          3163   
1    Offense        F   vs Lugano          8836          2881   
2    Offense        F   vs Lugano          8836          3586   
3    Offense        F   vs Lugano          8836          3881   
4    Defense        D   vs Lugano          8836           354   

   Time on Ice (s)  High Metabolic Power Distance (m) Mechanical Intensity  
0             1038                               1287                45,94  
1              689                               1724                42,85  
2              932                               2090                46,12  
3             1043                               1956                44,38  
4               92                                179                44,95  
     Group name Position Description  Duration (s)  Distance (m)  \
0       Defense        D   vs Zurich          7868          3140   
1       Off

In [1057]:
# Check if any NA in Each Row and column of the dataframes
print(df_imp_lugano.isnull().sum(axis=0))
# df_imp_lugano.isna().any(axis=1)

# We see there are no NA values in the match dataframe

print(
    "============================================================================================================="
)
# Check if any NA in Each Column of the dataframe
print(df_imp_zsc.isnull().sum(axis=0))

# We see 14 and 7, NA values in the column "Mechanical Intensity" in the two dataframes respectively

Group name                            0
Position                              0
Description                           0
Duration (s)                          0
Distance (m)                          0
Time on Ice (s)                       0
High Metabolic Power Distance (m)     0
Mechanical Intensity                 14
dtype: int64
Group name                           0
Position                             0
Description                          0
Duration (s)                         0
Distance (m)                         0
Time on Ice (s)                      0
High Metabolic Power Distance (m)    0
Mechanical Intensity                 7
dtype: int64


In [1058]:
# Remove the rows with NA values in the column "Mechanical Intensity"
df_imp_lugano = df_imp_lugano.dropna(subset=["Mechanical Intensity"])

df_imp_zsc = df_imp_zsc.dropna(subset=["Mechanical Intensity"])

# Check if any NA in Each Column of the dataframe
print(df_imp_lugano.isnull().sum(axis=0))
print(df_imp_lugano.shape)

print(
    "============================================================================================================="
)

print(df_imp_zsc.isnull().sum(axis=0))
print(df_imp_zsc.shape)

Group name                           0
Position                             0
Description                          0
Duration (s)                         0
Distance (m)                         0
Time on Ice (s)                      0
High Metabolic Power Distance (m)    0
Mechanical Intensity                 0
dtype: int64
(89, 8)
Group name                           0
Position                             0
Description                          0
Duration (s)                         0
Distance (m)                         0
Time on Ice (s)                      0
High Metabolic Power Distance (m)    0
Mechanical Intensity                 0
dtype: int64
(161, 8)


#### Creating a function to remove irrelevant rows or 'Players'

In [1059]:
# Function to remove rows where the value of column Group name is not equal to 'Offense' or 'Defense'
# This is done to remove the rows for the guest team present in the ZSC dataset


def filter_group_name(df):
    """
    Filters the DataFrame to retain rows where 'Group name' is 'Offense' or 'Defense'.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.

    Returns:
    - pd.DataFrame: The filtered DataFrame.
    """
    return df[df["Group name"].isin(["Offense", "Defense", "Goalies"])]


# Example usage:
df_filtered_zsc = filter_group_name(df_imp_zsc)

df_filtered_lugano = filter_group_name(df_imp_lugano)

# Display the filtered data:
df_filtered_zsc

# Display the filtered data:
# df_filtered_lugano

,Group name,Position,Description,Duration (s),Distance (m),Time on Ice (s),High Metabolic Power Distance (m),Mechanical Intensity
0,Defense,D,vs Zurich,7868,3140,1144,1223,"38,62"
1,Offense,F,vs Zurich,7868,3037,832,1359,"29,67"
2,Offense,F,vs Zurich,7868,4872,1394,2227,"37,04"
10,Offense,F,vs Zurich,7868,3648,1028,1591,"38,46"
11,Offense,F,vs Zurich,7868,2747,713,1326,"39,73"
...,...,...,...,...,...,...,...,...
163,Offense,F,Period 3,1854,930,236,219,"16,75"
164,Offense,F,Period 3,1854,1218,380,448,"37,02"
165,Offense,F,Period 3,1854,927,284,207,"23,5"
166,Offense,F,Period 3,1854,879,271,310,"34,28"


#### Adding the Player ID only to the rows with Column 'Description' = 'vs Lugano' or 'vs Zurich'

In [1060]:
def add_player_id(df):
    """
    Adds a new column 'Player ID' to the DataFrame starting from 1, but only to the rows
    with 'Description' as 'vs Lugano' or 'vs Zurich'.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.

    Returns:
    - pd.DataFrame: The DataFrame with the new 'Player ID' column added to specific rows.
    """
    mask = df["Description"].isin(["vs Lugano", "vs Zurich"])
    df.loc[mask, "Player ID"] = list(range(1, sum(mask) + 1))
    df["Player ID"] = df["Player ID"].astype(
        "Int64"
    )  # Using Int64 to handle potential NaNs
    return df


# Example usage:
df_imp_lugano = add_player_id(df_filtered_lugano)
df_imp_zsc = add_player_id(df_filtered_zsc)

# Displaying the first 7 rows:
print(df_imp_lugano.head(7))

print(
    "============================================================================================================="
)

print(df_imp_zsc.head(7))

  Group name    Position Description  Duration (s)  Distance (m)  \
0    Defense           D   vs Lugano          8836          3163   
1    Offense           F   vs Lugano          8836          2881   
2    Offense           F   vs Lugano          8836          3586   
3    Offense           F   vs Lugano          8836          3881   
4    Defense           D   vs Lugano          8836           354   
5    Defense           D   vs Lugano          8836          5087   
6    Goalies  Goalkeeper   vs Lugano          8836          1066   

   Time on Ice (s)  High Metabolic Power Distance (m) Mechanical Intensity  \
0             1038                               1287                45,94   
1              689                               1724                42,85   
2              932                               2090                46,12   
3             1043                               1956                44,38   
4               92                                179            

C:\Users\samue\AppData\Local\Temp\ipykernel_19652\1598695549.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\samue\AppData\Local\Temp\ipykernel_19652\1598695549.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [1061]:
def add_player_id(df):
    """
    Adds a new column 'Player ID' to the DataFrame starting from 1.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.

    Returns:
    - pd.DataFrame: The DataFrame with the new 'Player ID' column.
    """
    df.insert(loc=0, column="Player ID", value=range(1, 1 + len(df)))
    return df


# Example usage:
df_imp_lugano = add_player_id(df_filtered_lugano)
df_imp_zsc = add_player_id(df_filtered_zsc)

# Displaying the first 7 rows:
print(df_imp_lugano.head(7))

print(
    "============================================================================================================="
)

print(df_imp_zsc.head(7))

ValueError: cannot insert Player ID, already exists

#### Check the data types of the columns

In [ ]:
# Check the data types of the columns
print(df_imp_lugano.dtypes)

print(
    "============================================================================================================="
)

print(df_imp_zsc.dtypes)

#### Convert the data type of the column mechanical intensity from string to float

In [ ]:
def convert_mechanical_intensity_to_float(df):
    """
    Converts the data type of the 'Mechanical Intensity' column from string to float.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.

    Returns:
    - pd.DataFrame: The DataFrame with the 'Mechanical Intensity' column converted to float.
    """
    df["Mechanical Intensity"] = (
        df["Mechanical Intensity"].str.replace(",", ".").astype(float)
    )
    return df


# Example usage:
df_imp_lugano = convert_mechanical_intensity_to_float(df_imp_lugano)
df_imp_zsc = convert_mechanical_intensity_to_float(df_imp_zsc)

# Checking the data types:
print(df_imp_lugano.dtypes)

print(
    "============================================================================================================="
)

print(df_imp_zsc.dtypes)

#### Handling missing values

### CONCEPTUAL FRAMEWORK AND METHODOLOGY

Given the anonymized nature and the small sample size of our dataset, combined with numerous variables at play, we've made certain assumptions to ensure its suitable for our use-case. Here’s our approach to understanding player fatigue:

1. **Picking Player Data:**
   Due to lack of player identification data, we had to infer that first 21 rows represent individual player data. This is based on the understanding that a hockey team typically consists of 20 players. This selection ensures we capture data for each unique player.

2. **Identifying Player Roles:**
   We assess the 'Position' column to differentiate between a forward (FW), defenseman (D) and Goalkeeper. The roles they play on the ice rink might result in different fatigue levels.

3. **Data Segmentation:**
   With hundreds of variables present, we narrow down and segment data by recommended metrics from the icehockey club. This targeted analysis aids in the precise estimation of the Fatigue Index (FI).

4. **Developing the Fatigue Formula:**
   Using a combination of the dataset metrics, we create a formula that captures the aspects of player fatigue. This formula is designed to be sensitive to player roles, game dynamics, and other key factors.

5. **Fatigue Estimation:**
   Using the curated data from the previous steps, we derive the Fatigue Index (FI).

6. **Visualization:**
   We design a clear and easy-to-understand graphic to show the fatigue levels. This helps coaches quickly see and understand player tiredness.


#### Create a function to filter the dataset based on the Period or the full game

In [ ]:
# As we want only unique player data, we can safely select the game data i.e rows with Description = "vs Lugano"
# We can infer that because an ice hockey team typically consists of 20 players, including 2 goaltenders and 18 skaters.
# and "vs Lugano" implies that that row is for the match data NOT period data.
# and the proportion of forwards to defensemen


def filter_period_data(df, description):
    """
    Filters the DataFrame based on the given description.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - description (str): The description to filter by. Can be '1st Period', '2nd Period', '3rd Period' or 'vs Lugano' or 'vs Zurich'.

    Returns:
    - pd.DataFrame: The filtered DataFrame.
    """
    return df[df["Description"] == description]


# Example usage:
df_imp_lugano_game = filter_period_data(df_imp_lugano, "vs Lugano")
df_imp_zsc_game = filter_period_data(df_imp_zsc, "vs Zurich")

# Display the filtered data:
# print(df_imp_lugano_match)

print(
    "============================================================================================================="
)

print(df_imp_zsc_game)

print(df_imp_lugano_game)

#### Drop the rows with NA values


In [ ]:
# Inspect the rows with NA values in the column "Mechanical Intensity"
print(df_imp_lugano[df_imp_lugano["Mechanical Intensity"].isna()])

# We can safely drop these rows as we clearly see no metric values are present in these rows
df_imp_lugano.dropna(inplace=True)

In [ ]:
# Calculate the ratio of forwards to defensemen using position column

df_imp_lugano_match["Position"].value_counts()

In [ ]:
# Calculate ratio of F to D
F = df_imp_lugano_match["Position"].value_counts()[0]
D = df_imp_lugano_match["Position"].value_counts()[1]
ratio = F / D

# print ratio as a fraction

print(ratio)

### CREATING A RELATIVE FATIGUE LEVEL FORMULA

- Typically involves empirical research, biomechanics, and physiological considerations.
- However, we propose a simple, hypothetical formula to estimate fatigue.
- DISCLAIMER : This is a rough estimation and not based on scientific research.
- Using the important metrics recommended by the Ice Hockey Club & Professor Martin Rumo, we attempt to create a formula that can be used to estimate fatigue levels.


#### Relative Fatigue Index (RFI) = [A * (Time on ice/ Match Duration) + B * (High Metabolic Power Distance / Total Distance) + C * (Mechanical Intensity)]

- TIME FACTOR : Percentage of Time spent on Ice. Here, the idea is simple: the more time a player spends on the ice, the more fatigued they're likely to be.

- DISTANCE OR INTENSITY FACTOR : Percentage of the distance covered at high intensities. It's a straightforward indicator of how hard a player pushed themselves relative to their overall activity. It is the distance covered at high metabolic power divided by the total distance covered.

- MECHANICAL OR LOAD INTENSITY : How intense a player's session was concerning accelerations and decelerations.It models the intensity of a session or phase and provides important information about its impact on the lower limbs. It is the Mechanical Load divided by the total time in minutes.

- A, B, and C: Weights assigned to each of the factors. These weights are determined by the coach based on the importance of each factor in the context of the team's strategy and the coach's philosophy.


### FOR MATCH DATA


#### NORMALIZE MECHANICAL INTENSITY (MATCH DATA)


In [ ]:
# First we need to normalize the Mechanical Intensity column, so that the values are between 0 and 1
# This is to make sure that it can be added to TIME FACTOR, and DISTANCE FACTOR which are ratios in the next step
# With historical data, from the clubs, min and max intensities can be adjusted further in the future to normalize the intensity

min_val = df_imp_lugano_match["Mechanical Intensity"].min()
max_val = df_imp_lugano_match["Mechanical Intensity"].max()

df_imp_lugano_match["Normalized Mechanical Intensity"] = (
    df_imp_lugano_match["Mechanical Intensity"] - min_val
) / (max_val - min_val)

df_imp_lugano_match

#### CREATE RELATIVE FATIGUE INDEX (RFI) FUNCTION


In [ ]:
def fatigue_index(df, A=1, B=1, C=1):
    """This function calculates the fatigue index for each player."""
    return (
        A * (df["Time on Ice (s)"] / df["Duration (s)"])
        + B * df["High Metabolic Power Distance (m)"] / df["Distance (m)"]
    ) + C * df["Normalized Mechanical Intensity"]


# test the function
A = 1
B = 1
C = 1

fatigue_index(df_imp_lugano_match, A, B, C)

#### FATIGUE INDEX CALCULATION (MATCH DATA)


In [ ]:
# 1. Compute the intermediate fatigue index for each row
df_imp_lugano_match["Fatigue Index"] = df_imp_lugano_match.apply(fatigue_index, axis=1)

# 2. Normalize the fatigue index
min_fatigue = df_imp_lugano_match["Fatigue Index"].min()
max_fatigue = df_imp_lugano_match["Fatigue Index"].max()

# +1 to make sure the values are between 1 and 2
df_imp_lugano_match["Normalized Fatigue Index"] = (
    df_imp_lugano_match["Fatigue Index"] - min_fatigue
) / (max_fatigue - min_fatigue) + 1

# Sort the dataframe by the final fatigue index in descending order
# This will give us the most fatigued player at the top and the least fatigued player at the bottom

df_imp_lugano_match_sorted = df_imp_lugano_match.sort_values(
    by="Normalized Fatigue Index", ascending=True
)

df_imp_lugano_match_sorted

#### CREATE FUNCTION TO CALCULATE THE FATIGUE INDEX FOR EACH PLAYER


#### COLOR CATEGORIZATION OF FATIGUE LEVELS


In [ ]:
# MANUAL THRESHOLD BASED CATEGORIZATION OF FATIGUE INDEX

# Fixed thresholds
high_threshold = 1.57
low_threshold = 1


# Categorize values based on manual thresholds
df_imp_lugano_match_sorted["Fatigue Category"] = pd.cut(
    df_imp_lugano_match_sorted["Fatigue Index"],
    [0, low_threshold, high_threshold, 2],  # Explicitly using 2 as max value
    labels=["Low", "Medium", "High"],
    include_lowest=True,
)

# Display the results

df_imp_lugano_match_sorted.head(
    21
)  # [["Player ID", "Relative Fatigue Index", "Fatigue Category"]]

#### VISUALIZATION OF FATIGUE INDEX FOR EACH PLAYER


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Defining the color palette for the bars based on fatigue categories
color_map = {
    "Low": "#66FF66",  # Darker Green
    "Medium": "#FFFF66",  # Darker Yellow
    "High": "#FF6666",  # Darker Red
}
colors = df_imp_lugano_match_sorted["Fatigue Category"].map(color_map)

# Plotting the data
sns.set_style("whitegrid")
plt.figure(figsize=(14, 8))

# Using the 'order' parameter to ensure bars are plotted in the correct ascending order
bar_plot = sns.barplot(
    x="Player ID",
    y="Normalized Fatigue Index",
    data=df_imp_lugano_match_sorted,
    palette=colors,
    order=df_imp_lugano_match_sorted["Player ID"],
)
plt.title("Normalized Fatigue Index of Players", fontsize=18)
plt.xlabel("Player ID", fontsize=14)
plt.ylabel("Normalized Fatigue Index", fontsize=14)
plt.yticks([i for i in range(1, 3)], fontsize=14)
plt.xticks(fontsize=14)
plt.ylim(0.75, 2.25)
bar_plot.set_yticks([i * 0.25 + 0.75 for i in range(6)])

# Add legend
for category, color in color_map.items():
    plt.plot([], [], " ", label=category, color=color, marker="s", markersize=10)
plt.legend(
    title="Fatigue Category",
    loc="upper center",
    fontsize=12,
    title_fontsize=14,
    ncol=3,
    columnspacing=26,
)

# Add values of "Position" above the x-axis
for index, position in enumerate(df_imp_lugano_match_sorted["Position"]):
    plt.text(
        index, 0.8, position, ha="center", fontsize=12, color="black", va="baseline"
    )


# Displaying the plot
plt.tight_layout()
plt.show()

### ANALYSIS OF AVERAGE FATIGUE INDEX OF TEAM BY PERIODS


#### PERIOD 1 : FATIGUE INDEX CALCULATION


In [ ]:
# filter row by Description column i.e. Periods of the game : Period 1, Period 2, Period 3, Period 4


df_period1 = df_imp_lugano[df_imp_lugano["Description"].str.contains("Period 1")]

# Normalize the Mechanical Intensity column, so that the values are between 0 and 1

min_val = df_period1["Mechanical Intensity"].min()
max_val = df_period1["Mechanical Intensity"].max()

df_period1["Normalized Mechanical Intensity"] = (
    df_period1["Mechanical Intensity"] - min_val
) / (max_val - min_val)


# 1. Compute the  fatigue index for each row
df_period1["Fatigue Index"] = df_period1.apply(fatigue_index, axis=1)

# 2. Normalize the fatigue index
min_fatigue = df_period1["Fatigue Index"].min()
max_fatigue = df_period1["Fatigue Index"].max()

# +1 to make sure the values are between 1 and 2
df_period1["Normalized Fatigue Index"] = (df_period1["Fatigue Index"] - min_fatigue) / (
    max_fatigue - min_fatigue
) + 1

# Sort the dataframe by the final fatigue index in descending order
# This will give us the most fatigued player at the top and the least fatigued player at the bottom

df_period1_sorted = df_period1.sort_values(
    by="Normalized Fatigue Index", ascending=True
)


# Calculate the average fatigue index of the players in period 1

avg_FI_period1 = df_period1_sorted["Fatigue Index"].mean()

print(avg_FI_period1)

df_period1_sorted

#### PERIOD 2 : FATIGUE INDEX CALCULATION


In [ ]:
df_period2 = df_imp_lugano[df_imp_lugano["Description"].str.contains("Period 2")]

# Normalize the Mechanical Intensity column, so that the values are between 0 and 1

min_val = df_period2["Mechanical Intensity"].min()
max_val = df_period2["Mechanical Intensity"].max()

df_period2["Normalized Mechanical Intensity"] = (
    df_period2["Mechanical Intensity"] - min_val
) / (max_val - min_val)


df_period2["Fatigue Index"] = df_period2.apply(fatigue_index, axis=1)

# 2. Normalize the fatigue index
min_fatigue = df_period2["Fatigue Index"].min()
max_fatigue = df_period2["Fatigue Index"].max()

# +1 to make sure the values are between 1 and 2
df_period2["Normalized Fatigue Index"] = (df_period2["Fatigue Index"] - min_fatigue) / (
    max_fatigue - min_fatigue
) + 1

# Sort the dataframe by the final fatigue index in descending order
# This will give us the most fatigued player at the top and the least fatigued player at the bottom

df_period2_sorted = df_period2.sort_values(
    by="Normalized Fatigue Index", ascending=True
)


# Calculate the average fatigue index of the players in period 2

avg_FI_period2 = df_period2_sorted["Fatigue Index"].mean()

print(avg_FI_period2)

df_period2_sorted

#### PERIOD 3 : FATIGUE INDEX CALCULATION


In [ ]:
df_period3 = df_imp_lugano[df_imp_lugano["Description"].str.contains("Period 3")]

# Normalize the Mechanical Intensity column, so that the values are between 0 and 1

min_val = df_period3["Mechanical Intensity"].min()
max_val = df_period3["Mechanical Intensity"].max()

df_period3["Normalized Mechanical Intensity"] = (
    df_period3["Mechanical Intensity"] - min_val
) / (max_val - min_val)

df_period3

df_period3["Fatigue Index"] = df_period3.apply(fatigue_index, axis=1)

# 2. Normalize the fatigue index
min_fatigue = df_period3["Fatigue Index"].min()
max_fatigue = df_period3["Fatigue Index"].max()

# +1 to make sure the values are between 1 and 2
df_period3["Normalized Fatigue Index"] = (df_period3["Fatigue Index"] - min_fatigue) / (
    max_fatigue - min_fatigue
) + 1

# Sort the dataframe by the final fatigue index in descending order
# This will give us the most fatigued player at the top and the least fatigued player at the bottom

df_period3_sorted = df_period3.sort_values(
    by="Normalized Fatigue Index", ascending=True
)

# Calculate the average fatigue index of the players in period 3

avg_FI_period3 = df_period3_sorted["Fatigue Index"].mean()

print(avg_FI_period3)

df_period3_sorted

#### OVERTIME : FATIGUE INDEX CALCULATION


In [ ]:
df_overtime = df_imp_lugano[df_imp_lugano["Description"].str.contains("Overtime")]

# Normalize the Mechanical Intensity column, so that the values are between 0 and 1

min_val = df_overtime["Mechanical Intensity"].min()
max_val = df_overtime["Mechanical Intensity"].max()

df_overtime["Normalized Mechanical Intensity"] = (
    df_overtime["Mechanical Intensity"] - min_val
) / (max_val - min_val)

df_overtime

df_overtime["Fatigue Index"] = df_overtime.apply(fatigue_index, axis=1)

# 2. Normalize the fatigue index
min_fatigue = df_overtime["Fatigue Index"].min()
max_fatigue = df_overtime["Fatigue Index"].max()

# +1 to make sure the values are between 1 and 2
df_overtime["Normalized Fatigue Index"] = (
    df_overtime["Fatigue Index"] - min_fatigue
) / (max_fatigue - min_fatigue) + 1

# Sort the dataframe by the final fatigue index in descending order
# This will give us the most fatigued player at the top and the least fatigued player at the bottom

df_overtime_sorted = df_overtime.sort_values(
    by="Normalized Fatigue Index", ascending=True
)


# Calculate the average fatigue index of the players in overtime

avg_FI_overtime = df_overtime_sorted["Fatigue Index"].mean()

print(avg_FI_overtime)

df_overtime_sorted

### LINE CHART OF AVERAGE FATIGUE INDEX OF TEAM (CHANGE IN FATIGUE INDEX OVER TIME)

In [ ]:
import plotly.graph_objects as go

# Create a list of the average fatigue index of players in each period and overtime
avg_FI = [avg_FI_period1, avg_FI_period2, avg_FI_period3, avg_FI_overtime]

# Create a list of the periods and overtime
periods = ["Period 1", "Period 2", "Period 3", "Overtime"]

# Create a Plotly figure for an interactive line chart
fig = go.Figure(
    data=go.Scatter(
        x=periods, y=avg_FI, mode="lines+markers", marker=dict(color="green")
    )
)

# Customize the chart layout
fig.update_layout(
    title="Average Fatigue Index of Team Lugano in each Period",
    xaxis=dict(title="Periods"),
    yaxis=dict(title="Average Fatigue Index"),
)

# Show the interactive chart
fig.show()

In [ ]:
import plotly.graph_objects as go

# Create a list of the average fatigue index of players in each period and overtime
avg_FI = [avg_FI_period1, avg_FI_period2, avg_FI_period3, avg_FI_overtime]

# Create a list of the periods and overtime
periods = ["Period 1", "Period 2", "Period 3", "Overtime"]

# Calculate cumulative values for the y-axis
cumulative_avg_FI = [sum(avg_FI[: i + 1]) for i in range(len(avg_FI))]

# Create a Plotly figure for an interactive line chart
fig = go.Figure(
    data=go.Scatter(
        x=periods, y=cumulative_avg_FI, mode="lines+markers", marker=dict(color="green")
    )
)

# Customize the chart layout
fig.update_layout(
    title="Cumulative - Average Fatigue Index of Team Lugano",
    xaxis=dict(title="Periods"),
    yaxis=dict(title="Cumulative Average Fatigue Index"),
)

# Show the interactive chart
fig.show()